In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
import time 
from datetime import datetime
import requests
from config import weather_api_key

In [2]:
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key


In [3]:
# Create a set of random latitude and longitude combinations

lats = np.random.uniform(low = -90.000, high = 90.000, size = 1500)
lngs = np.random.uniform(low = -180.000, high = 180.000, size = 1500)

lat_lngs = zip(lats, lngs)
lat_lngs

In [4]:
# Add the latitudes and longitudes to a list

coordinates = list(lat_lngs)

In [5]:
# Print the combinations
# for coordinate in coordinates:

#     print(coordinate[0], coordinate[1])

In [6]:
# Create a new list for holding cities
cities = []

# ID the nearest city for each lat/long combination
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list

    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

642

In [7]:
# Create an empty list to hold the weather data
city_data = []

# Print the beginning of the logging.

print("Beginning Data Retrieval       ")
print("-------------------------------")

# Create counters

record_count = 1
set_count = 1

# Loop through all of the cities in the list
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(1)

    # Create endpoint URL with each city
    city_url = url + "&q=" + cities[i]


    # Log the URL, record, and set numbers and the city

    print(f"Processing Record {record_count} of Set {set_count} | {city}")

    # Add 1 to the record count

    record_count += 1

# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(
            city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")


Beginning Data Retrieval       
-------------------------------
Processing Record 1 of Set 1 | yellowknife
Processing Record 2 of Set 1 | rikitea
Processing Record 3 of Set 1 | bathsheba
Processing Record 4 of Set 1 | aksu
Processing Record 5 of Set 1 | solnechnyy
Processing Record 6 of Set 1 | padang
Processing Record 7 of Set 1 | pangody
Processing Record 8 of Set 1 | konosha
Processing Record 9 of Set 1 | lerwick
Processing Record 10 of Set 1 | williams lake
Processing Record 11 of Set 1 | vaini
Processing Record 12 of Set 1 | grindavik
Processing Record 13 of Set 1 | pisco
Processing Record 14 of Set 1 | ushuaia
Processing Record 15 of Set 1 | punta arenas
Processing Record 16 of Set 1 | kuruman
Processing Record 17 of Set 1 | sampit
Processing Record 18 of Set 1 | saldanha
Processing Record 19 of Set 1 | ilulissat
Processing Record 20 of Set 1 | umkomaas
Processing Record 21 of Set 1 | haines junction
Processing Record 22 of Set 1 | rundu
Processing Record 23 of Set 1 | college
Pr

In [8]:
# Convert the array of dictionaries to a pandas DF

city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Yellowknife,62.4560,-114.3525,64.92,34,40,11.50,CA,2022-06-27 18:27:19
1,Rikitea,-23.1203,-134.9692,73.63,68,4,7.45,PF,2022-06-27 18:20:35
2,Bathsheba,13.2167,-59.5167,86.61,74,40,23.02,BB,2022-06-27 18:27:32
3,Aksu,41.1231,80.2644,87.49,20,15,4.27,CN,2022-06-27 18:27:32
4,Solnechnyy,50.7214,136.6319,56.23,94,16,2.51,RU,2022-06-27 18:27:33
5,Padang,-0.9492,100.3543,78.30,87,99,2.06,ID,2022-06-27 18:27:33
6,Pangody,65.8500,74.4859,57.42,80,78,17.74,RU,2022-06-27 18:27:34
7,Konosha,60.9736,40.2570,55.63,63,85,4.76,RU,2022-06-27 18:27:34
8,Lerwick,60.1545,-1.1494,55.40,90,71,11.56,GB,2022-06-27 18:27:35
9,Williams Lake,52.1415,-122.1445,83.05,34,40,6.91,CA,2022-06-27 18:27:35


In [9]:
new_order = ["City", "Country", "Date", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed"]
city_data_df = city_data_df[new_order]
city_data_df.head()


,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Yellowknife,CA,2022-06-27 18:27:19,62.4560,-114.3525,64.92,34,40,11.50
1,Rikitea,PF,2022-06-27 18:20:35,-23.1203,-134.9692,73.63,68,4,7.45
2,Bathsheba,BB,2022-06-27 18:27:32,13.2167,-59.5167,86.61,74,40,23.02
3,Aksu,CN,2022-06-27 18:27:32,41.1231,80.2644,87.49,20,15,4.27
4,Solnechnyy,RU,2022-06-27 18:27:33,50.7214,136.6319,56.23,94,16,2.51


In [10]:
# Create the output file (CSV)
output_data_file = "Weather_Data/cities.csv"

# Export the City_Data into csv
city_data_df.to_csv(output_data_file, index_label="City_ID")